In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from math import floor, ceil
%matplotlib inline

In [2]:
df = pd.read_csv('train.csv', index_col=0)

In [3]:
df.drop(axis=1,columns='color', inplace=True)

In [4]:
X = df.drop(axis=1, columns='type')
Y= pd.get_dummies(df['type'], drop_first = True)

In [5]:
#Simple NN: Input -> hidden layer (activation function ReLU) -> output (softmax) + Dropout to avoid overfitting
def multilayer_perceptron(x, weights, biases, keep_prob):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.dropout(layer_1, keep_prob)
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    print(out_layer)
    return out_layer


In [6]:
n_hidden_1 = 13 ##number of neurons
n_input = X.shape[1]
n_classes = Y.shape[1]

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

keep_prob = tf.placeholder("float")

In [7]:
training_epochs = 5000
display_step = 1000
batch_size = 32

x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [8]:
predictions = multilayer_perceptron(x, weights, biases, keep_prob)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.00001).minimize(cost)

Tensor("add:0", shape=(?, 2), dtype=float32)


In [9]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = int(len(X) / batch_size)
        x_batches = np.array_split(X, total_batch)
        y_batches = np.array_split(Y, total_batch)
        for i in range(total_batch):
            batch_x, batch_y = x_batches[i], y_batches[i]
            _, c = sess.run([optimizer, cost], 
                            feed_dict={
                                x: batch_x, 
                                y: batch_y, 
                                keep_prob: 0.8
                            })
            avg_cost += c / total_batch
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")
    correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: X, y: Y, keep_prob: 1.0}))

Epoch: 0001 cost= 0.513384770
Epoch: 1001 cost= 0.502915572
Epoch: 2001 cost= 0.512463445
Epoch: 3001 cost= 0.496136329
Epoch: 4001 cost= 0.509713785
Optimization Finished!
Accuracy: 0.619946
